#Draft


In [3]:
# from datasets import load_dataset

# ds = load_dataset("phantatbach/TMed")

In [4]:
# print(ds)
# print(ds["train"].features)
# print(ds['train'][0])

In [5]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("xiaoweixumedicalai/cardiacudc-dataset")

# print("Path to dataset files:", path)

In [6]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("shoybhasan/camus-human-heart-data")

# print("Path to dataset files:", path)

In [7]:
# !file /kaggle/input/camus-human-heart-data/download
# !ls
# !mv download camus.zip
# !unzip camus.zip -d camus_dataset


In [8]:
# !zip -r output_filename.zip /root/.cache/kagglehub/datasets

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

# !cp output_filename.zip /content/drive/MyDrive/


In [10]:
# import urllib.parse
# import IPython.display as display

# file_path = '/content/output_filename.zip'
# file_name = file_path.split('/')[-1]

# # Encode the path for the link
# encoded_path = urllib.parse.quote(file_path)

# # This is the actual temporary download URL Colab uses
# url = f'/download/{encoded_path}'

# # Show it as a clickable link
# display.HTML(f'<a href="{url}" target="_blank">Click here for direct download</a>')


In [11]:
# !wget -r -N -c -np --user mehran8 --ask-password https://physionet.org/files/mimic-iv-echo/0.1/


# HVC

Downloading the data

In [12]:
import requests
import zipfile
import os

def download_file(url, extract_to="."):
    filename = url.split("/")[-1]  # get filename from URL

    print(f"Downloading {filename}...")
    response = requests.get(url)
    response.raise_for_status()  # ensure download was successful

    # Save the file
    with open(filename, "wb") as f:
        f.write(response.content)

    # If it's a ZIP, extract it
    if filename.endswith(".zip"):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted to {extract_to}")
    else:
        print(f"Saved {filename}")

# Example usage (replace with your actual URLs)
url1 = "https://data.unityimaging.net/files/2020-12-05/png-cache.zip"
url2 = "https://data.unityimaging.net/files/additional/view_index_20220705.csv"

download_file(url1, extract_to="data1")
download_file(url2, extract_to="data2")


Extracting png-cache.zip...
Extracted to data1
Saved view_index_20220705.csv


Preprocessing

In [13]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

root_dir = "/content/data1/png-cache/01"   # Unity dataset root
csv_file = "/content/view_index_20220705.csv"
output_dir = "dataset_cleaned"
os.makedirs(output_dir, exist_ok=True)

df = pd.read_csv(csv_file)
label_map = dict(zip(df['file'], df['view']))  # adjust if columns differ

target_size = (224, 224)

for fname, view in tqdm(label_map.items()):
    # locate file in nested folders
    sub1, sub2 = fname[3:5], fname[5:7]
    fpath = os.path.join(root_dir, sub1, sub2, fname)
    if not os.path.exists(fpath):
        # print(f"missing: {fpath}")
        continue  # skip if file missing

    # open
    img = Image.open(fpath).convert("RGB")
    w, h = img.size

    # dynamic crop (50 top/bottom, 100 left/right)
    left, top, right, bottom = 100, 50, w-100, h-50
    img = img.crop((left, top, right, bottom))

    # resize
    img = img.resize(target_size)

    # save in clean folder
    outdir = os.path.join(output_dir, view)
    os.makedirs(outdir, exist_ok=True)
    img.save(os.path.join(outdir, fname))


100%|██████████| 15500/15500 [02:33<00:00, 101.23it/s]


Splitting the dataset

In [14]:
import os
import shutil
import random

# --- Paths ---
dataset_dir = "/content/dataset_cleaned"   # current dataset (all images per class)
output_dir = "dataset_split"      # new split dataset

# --- Split ratios ---
train_ratio, val_ratio, test_ratio = 0.7, 0.15, 0.15

# --- Fix random seed for reproducibility ---
random.seed(42)

# --- Create output structure ---
for split in ["train", "val", "test"]:
    for cls in os.listdir(dataset_dir):
        os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)

# --- Split images ---
for cls in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, cls)
    if not os.path.isdir(class_dir):
        continue

    images = os.listdir(class_dir)
    random.shuffle(images)

    n_total = len(images)
    n_train = int(train_ratio * n_total)
    n_val = int(val_ratio * n_total)

    train_files = images[:n_train]
    val_files = images[n_train:n_train+n_val]
    test_files = images[n_train+n_val:]

    # Move files
    for fname in train_files:
        shutil.copy(os.path.join(class_dir, fname), os.path.join(output_dir, "train", cls, fname))
    for fname in val_files:
        shutil.copy(os.path.join(class_dir, fname), os.path.join(output_dir, "val", cls, fname))
    for fname in test_files:
        shutil.copy(os.path.join(class_dir, fname), os.path.join(output_dir, "test", cls, fname))

    print(f"{cls}: {n_total} → train={len(train_files)}, val={len(val_files)}, test={len(test_files)}")

print("✅ Dataset split completed at:", output_dir)


a2c: 1029 → train=720, val=154, test=155
a4c: 3311 → train=2317, val=496, test=498
a5c: 144 → train=100, val=21, test=23
a3c: 147 → train=102, val=22, test=23
plax: 2891 → train=2023, val=433, test=435
✅ Dataset split completed at: dataset_split


# Training and Evaluating

Base modules

In [2]:
import tensorflow as tf
import os

# --- Dataset Loader ---
def load_datasets(dataset_dir, img_size=(224,224), batch_size=32):
    raw_train_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(dataset_dir, "train"),
        image_size=img_size,
        batch_size=batch_size
    )
    raw_val_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(dataset_dir, "val"),
        image_size=img_size,
        batch_size=batch_size
    )
    raw_test_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(dataset_dir, "test"),
        image_size=img_size,
        batch_size=batch_size
    )

    # ✅ Capture class names before wrapping
    class_names = raw_train_ds.class_names

    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = raw_train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = raw_val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    test_ds = raw_test_ds.cache().prefetch(buffer_size=AUTOTUNE)

    return train_ds, val_ds, test_ds, class_names


# --- Model Builder ---
def build_model(base_model_fn, img_size, num_classes, dropout_rate=0.2, learning_rate=1e-4):
    base_model = base_model_fn(
        input_shape=img_size + (3,),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False  # freeze

    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax")(x)

    model = tf.keras.Model(inputs=base_model.input, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


# --- Train + Export Pipeline ---
def train_and_export(model_name, base_model_fn, dataset_dir, out_prefix,
                     img_size=(224,224), batch_size=32, epochs=10):
    print(f"\n🔹 Training {model_name}...")
    train_ds, val_ds, test_ds, class_names = load_datasets(dataset_dir, img_size, batch_size)
    num_classes = len(class_names)
    print("Classes:", class_names)

    model = build_model(base_model_fn, img_size, num_classes)

    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

    test_loss, test_acc = model.evaluate(test_ds)
    print(f"✅ {model_name} Test Accuracy: {test_acc:.3f}")

    # Save and export
    tflite_out = f"{out_prefix}.tflite"

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    with open(tflite_out, "wb") as f:
        f.write(tflite_model)

    print(f"✅ {model_name} exported to {tflite_out}")
    return model, history


In [16]:
# import tensorflow as tf
# tf.debugging.set_log_device_placement(False)

Various Models

In [17]:
dataset_dir = "dataset_split"

train_and_export("ResNet50", tf.keras.applications.ResNet50, dataset_dir, "resnet50")


🔹 Training ResNet50...
Found 5262 files belonging to 5 classes.
Found 1126 files belonging to 5 classes.
Found 1134 files belonging to 5 classes.
Classes: ['a2c', 'a3c', 'a4c', 'a5c', 'plax']
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 47s 160ms/step - accuracy: 0.4487 - loss: 1.4958 - val_accuracy: 0.7806 - val_loss: 0.7629
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 55s 88ms/step - accuracy: 0.7108 - loss: 0.8094 - val_accuracy: 0.8073 - val_loss: 0.6025
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.7707 - loss: 0.6841 - val_accuracy: 0.8250 - val_loss: 0.5354
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 20s 89ms/step - accuracy: 0.8065 - loss: 0.5917 - val_accuracy: 0.8410 - val_loss: 0.4832
Epoch 5/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 90ms/step - accuracy: 0.8282 - loss: 0.5263 - val_accuracy: 0.8517 - val_loss: 0.4518
Epoch 6/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 15s 90ms/step - accuracy: 0.8410 - loss: 0.4811 - val_accuracy: 0

(<Functional name=functional, built=True>,
 <keras.src.callbacks.history.History at 0x7e3610118b00>)

In [18]:
train_and_export("DenseNet121", tf.keras.applications.DenseNet121, dataset_dir, "densenet121")


🔹 Training DenseNet121...
Found 5262 files belonging to 5 classes.
Found 1126 files belonging to 5 classes.
Found 1134 files belonging to 5 classes.
Classes: ['a2c', 'a3c', 'a4c', 'a5c', 'plax']
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 107s 426ms/step - accuracy: 0.3488 - loss: 2.8812 - val_accuracy: 0.5462 - val_loss: 1.3656
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 0.4446 - loss: 2.0275 - val_accuracy: 0.6581 - val_loss: 1.0180
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - accuracy: 0.5239 - loss: 1.6774 - val_accuracy: 0.7123 - val_loss: 0.8410
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.5833 - loss: 1.3869 - val_accuracy: 0.7513 - val_loss: 0.7567
Epoch 5/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 14s 82ms/step - accuracy: 0.6259 - loss: 1.2133 - val_accuracy: 0.7718 - val_loss: 0.6958
Epoch 6/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 14s 83ms/step - accuracy: 0.6438 - loss: 1.1217 - val_accurac

(<Functional name=functional_1, built=True>,
 <keras.src.callbacks.history.History at 0x7e3578147260>)

In [19]:
train_and_export("VGG16", tf.keras.applications.VGG16, dataset_dir, "vgg16") # seems to be unnecessary large with low accuracy


🔹 Training VGG16...
Found 5262 files belonging to 5 classes.
Found 1126 files belonging to 5 classes.
Found 1134 files belonging to 5 classes.
Classes: ['a2c', 'a3c', 'a4c', 'a5c', 'plax']
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 70s 251ms/step - accuracy: 0.2325 - loss: 5.7949 - val_accuracy: 0.4503 - val_loss: 1.7026
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 30s 185ms/step - accuracy: 0.4448 - loss: 2.1725 - val_accuracy: 0.5870 - val_loss: 1.3104
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 35s 213ms/step - accuracy: 0.5416 - loss: 1.7936 - val_accuracy: 0.6607 - val_loss: 1.0916
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 30s 184ms/step - accuracy: 0.6014 - loss: 1.5624 - val_accuracy: 0.7114 - val_loss: 0.9625
Epoch 5/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 30s 184ms/step - accuracy: 0.6345 - loss: 1.3655 - val_accuracy: 0.7398 - val_loss: 0.8683
Epoch 6/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 30s 182ms/step - accuracy: 0.6668 - loss: 1.2266 - val_accuracy:

(<Functional name=functional_2, built=True>,
 <keras.src.callbacks.history.History at 0x7e35786ba990>)

In [20]:
train_and_export("MobileNetV2", tf.keras.applications.MobileNetV2, dataset_dir, "mobilenetv2")


🔹 Training MobileNetV2...
Found 5262 files belonging to 5 classes.
Found 1126 files belonging to 5 classes.
Found 1134 files belonging to 5 classes.
Classes: ['a2c', 'a3c', 'a4c', 'a5c', 'plax']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 72s 185ms/step - accuracy: 0.4306 - loss: 1.3918 - val_accuracy: 0.6918 - val_loss: 0.9071
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.6347 - loss: 0.9750 - val_accuracy: 0.7353 - val_loss: 0.7894
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.6863 - loss: 0.8680 - val_accuracy: 0.7487 - val_loss: 0.7219
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.7174 - loss: 0.7896 - val_accuracy: 0.7700 - val_loss: 0.6747
Epoch 5/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.7347 - loss: 0.7420 - val_accuracy: 0.7815 - val_loss: 0.6425
Epoch 6/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.7525 - loss: 0.7185 - val_accuracy: 0.808

(<Functional name=functional_3, built=True>,
 <keras.src.callbacks.history.History at 0x7e33ea89acf0>)

In [4]:
dataset_dir = "dataset_split"

train_and_export("EfficientNetB0", tf.keras.applications.EfficientNetB0, dataset_dir, "efficientnetb0")


🔹 Training EfficientNetB0...
Found 5262 files belonging to 5 classes.
Found 1126 files belonging to 5 classes.
Found 1134 files belonging to 5 classes.
Classes: ['a2c', 'a3c', 'a4c', 'a5c', 'plax']
Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 79s 238ms/step - accuracy: 0.3576 - loss: 1.3984 - val_accuracy: 0.6625 - val_loss: 0.9975
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - accuracy: 0.6467 - loss: 1.0058 - val_accuracy: 0.7638 - val_loss: 0.8236
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - accuracy: 0.7424 - loss: 0.8352 - val_accuracy: 0.7895 - val_loss: 0.7222
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.7837 - loss: 0.7369 - val_accuracy: 0.8135 - val_loss: 0.6532
Epoch 5/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.8021 - loss: 0.6982 - val_accuracy: 0.8171 - val_loss: 0.6052
Epoch 6/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.8141 - loss: 0.6327 - val_accuracy: 0.8268 - val_loss: 0.5692
Epoch 7/10
165/165 ━━━━━━

(<Functional name=functional, built=True>,
 <keras.src.callbacks.history.History at 0x7b035499e570>)